### 1. One client to rule them all

The Dynamic Annotation Framework consists of a number of different services, each with a specific set of tasks that it can perform through REST endpoints. This module is designed to ease programmatic interaction with all of the various endpoints. Going forward, we also will be increasingly using authentication tokens for programmatic access to most if not all of the services. In order to collect a given server, dataset name, and user token together into a coherent package that can be used on multiple endpoints, we will use a FrameworkClient that can build appropriately configured clients for each of the specific services.

The following examples cover each of the client subtypes that are associated with a single service. The ImageryClient, which is a more complex collection of tools, will be covered elsewhere.

#### Initializing a FrameworkClient
Assuming that the services are on `www.dynamicannotationframework.com` and authentication tokens are either not being used or set up with default values (see next section), one needs only to specify the dataset name.

In [3]:
from annotationframeworkclient import FrameworkClient

dataset_name = 'pinky100'
client = FrameworkClient(dataset_name)

Just to confirm that this works, let's see if we can get the EM image source from the InfoService. If you get a reasonable looking path, everything is okay.

In [4]:
print(f"The image source is: {client.info.image_source()}")

The image source is: https://storage.googleapis.com/neuroglancer/pinky100_v0/son_of_alignment_v15_rechunked


### 2. Authentication Service

Going forward, we're going to need authentication tokens for programmatic access to our services. The AuthClient handles storing and loading your token or tokens and inserting it into requests in other clients.

We can access the auth client from `client.auth`. Once you have saved a token, you probably won't interact with this client very often, however it has some convenient features for saving new tokens the first time. Let's see if you have a token already. Probably not.

In [5]:
auth = client.auth
print(f"My current token is: {auth.token}")

My current token is: fake_token_123


#### Getting a new token
It is not yet possible to get a new token programmatically, but the function `get_new_token()` provides instructions for how to get and save it.

By default, the token is saved to `~/.cloudvolume/secrets/chunkedgraph-secret.json` as a string under the key `token`. The following steps will save a token there.

*Note: I am not sure where the auth server is being hosted right now, so we are going to use a fake token for documentation purposes*

In [6]:
auth.get_new_token(open=True)

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://www.dynamicannotationframework.com/auth/refresh_token
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token will invalidate the previous token!


In [7]:
new_token = 'fake_token_123'
auth.save_token(token=new_token, overwrite=True)
print(f"My token is now: {auth.token}")

My token is now: fake_token_123


#### Loading saved tokens
Try opening `~/.cloudvolume/secrets/chunkedgraph-secret.json` to see what we just created.

If we had wanted to use a different file or a different json key, we could have specified that in auth.save_token.

Because we used the default values, this token is used automatically when we intialize a new FrameworkClient. If we wanted to use a different token file, token key, or even directly specify a token we could do so here.

In [8]:
client = FrameworkClient(dataset_name)
print(f"Now my basic token is: {client.auth.token}")

client_direct = FrameworkClient(dataset_name, auth_token='another_fake_token_678')
print(f"A directly specified token is: {client_direct.auth.token}")

Now my basic token is: fake_token_123
A directly specified token is: another_fake_token_678


If you use a FrameworkClient, the AuthClient and its token will be automatically applied to any other services without further use. 

### 3. Info Service
A dataset has a number of complex paths to various data sources that together comprise a dataset. Rather than hardcode these paths, the InfoService allows one to query the location of each data source. This is also convenient in case data sources change.

An InfoClient is accessed at `client.info`.

In [9]:
client.info

In [10]:
client = FrameworkClient(dataset_name)
print(f"This is an info client for {client.info.dataset_name} on {client.info.server_address}")

This is an info client for pinky100 on https://www.dynamicannotationframework.com


#### Accessing dataset information
All of the information accessible for the dataset can be seen as a dict using `get_dataset_info()`.

In [11]:
client.info.get_dataset_info()

{'analysis_database_ip': 'www.dynamicannotationframework.com',
 'annotation_engine_endpoint': 'https://www.dynamicannotationframework.com/',
 'flat_segmentation_source': 'https://storage.googleapis.com/neuroglancer/pinky100_v0/seg/lost_no-random/bbox1_0',
 'graphene_source': 'graphene://https://www.dynamicannotationframework.com/segmentation/1.0/pinky100_sv16',
 'id': 1,
 'image_source': 'https://storage.googleapis.com/neuroglancer/pinky100_v0/son_of_alignment_v15_rechunked',
 'name': 'pinky100',
 'pychunkedgraph_supervoxel_source': 'https://storage.googleapis.com/neuroglancer/nkem/pinky100_v0/ws/lost_no-random/bbox1_0',
 'pychunkedgraph_viewer_source': 'https://www.dynamicannotationframework.com/segmentation/1.0/pinky100_sv16',
 'pychunkgraph_endpoint': 'http://35.237.202.194/',
 'pychunkgraph_segmentation_source': 'https://www.dynamicannotationframework.com/segmentation/1.0/pinky100_sv16',
 'synapse_segmentation_source': 'https://storage.googleapis.com/neuroglancer/pinky100_v0/clefts

Individual entries can be found as well. Use tab autocomplete to see the various possibilities.

In [12]:
client.info.graphene_source()

'graphene://https://www.dynamicannotationframework.com/segmentation/1.0/pinky100_sv16'

#### Adjusting formatting
Because of the way neuroglancer looks up data versus cloudvolume, sometimes one needs to convert between `gs://` style paths to `https://storage.googleapis.com/` stype paths. All of the path sources in the info client accept a `format_for` argument that can handle this, and correctly adapts to graphene vs precomputed data sources.

In [13]:
neuroglancer_style_source = client.info.image_source(format_for='neuroglancer')
print(f"With gs-style: { neuroglancer_style_source }")

cloudvolume_style_source = client.info.image_source(format_for='cloudvolume')
print(f"With https-style: { cloudvolume_style_source }")

With gs-style: precomputed://gs://neuroglancer/pinky100_v0/son_of_alignment_v15_rechunked
With https-style: precomputed://https://storage.googleapis.com/neuroglancer/pinky100_v0/son_of_alignment_v15_rechunked


### 4. JSON Service

We store the JSON description of a Neuroglancer state in a simple database at the JSON Service. This is a convenient way to build states to distribute to people, or pull states to parse work by individuals. The JSON Client is at `client.state`

In [15]:
client.state

#### Retrieving a state

JSON states are found simply by their ID, which you get when uploading a state. You can download a state with `get_state_json`.

In [18]:
example_id = 4845531975188480
example_state = client.state.get_state_json(example_id)
example_state['layers'][0]

{'source': 'precomputed://gs://microns-seunglab/minnie_v4/alignment/fine/sergiy_multimodel_v1/vector_fixer30_faster_v01/image_stitch_multi_block_v1',
 'type': 'image',
 'blend': 'default',
 'shaderControls': {},
 'name': 'Minnie65 (MIP2+)'}

#### Uploading a state
You can also upload states with `upload_state_json`. If you do this, the state id is returned by the function. Note that there is no easy way to query what you uploaded later, so be VERY CAREFUL with this state id if you wish to see it again.

*Note: If you are working with a Neuroglancer Viewer object or similar, in order to upload, use viewer.state.to_json() to generate this representation.*

In [19]:
example_state['layers'][0]['name'] = 'example_name'
new_id = client.state.upload_state_json(example_state)

In [20]:
test_state = client.state.get_state_json(new_id)
test_state['layers'][0]['name']

'example_name'

#### Generating Neuroglancer URLs

Neuroglancer can automatically look up a JSON state based on its ID if you pass the URL to it correctly. The function `build_neuroglancer_url` helps format these correctly. Note that you need to specify the base URL for the Neuroglancer deployment you wish to use.

In [21]:
url = client.state.build_neuroglancer_url(state_id=new_id, ngl_url='https://neuromancer-seung-import.appspot.com')
print(url)

https://neuromancer-seung-import.appspot.com/?json_url=https://www.dynamicannotationframework.com/nglstate/6553267771342848


### 5. ChunkedGraph

The ChunkedGraph client allows one to interact with the ChunkedGraph, which stores and updates the supervoxel agglomeration graph. This is most often useful for looking up an object root id of a supervoxel or looking up supervoxels belonging to a root id. The ChunkedGraph client is at `client.chunkedgraph`.

#### Look up a supervoxel
Usually in Neuroglancer, one never notices supervoxel ids, but they are important for programmatic work. In order to look up the root id for a location in space, one needs to use the supervoxel segmentation to get the associated supervoxel id. The ChunkedGraph client makes this easy using the `get_root_ids` method.

In [22]:
sv_id = 104200755619042523
client.chunkedgraph.get_root_id(supervoxel_id=sv_id)

array([648518346349541252], dtype=uint64)

However, as proofreading occurs, the root id that a supervoxel belongs to can change. By default, this function returns the current state, however one can also provide a UTC timestamp to get the root id at a particular moment in history. This can be useful for reproducible analysis. Note below that the root id for the same supervoxel is different than it is now.

In [23]:
import datetime

# I looked up the UTC POSIX timestamp from a day in early 2019. 
timestamp = datetime.datetime.utcfromtimestamp(1546595253)

sv_id = 104200755619042523
client.chunkedgraph.get_root_id(supervoxel_id=sv_id, timestamp=timestamp)

array([648518346348172238], dtype=uint64)

#### Getting supervoxels for a root id

A root id is associated with a particular agglomeration of supervoxels, which can be found with the `get_leaves` method. A new root id is generated for every new change in the chunkedgraph, so time stamps do not apply.

In [24]:
root_id = 648518346349541252
client.chunkedgraph.get_leaves(root_id)

array([ 84777895599276224,  84777895599276227,  84777895599276228, ...,
       104200755619042523, 104200755619042333, 104200755619042335],
      dtype=uint64)

### 7. AnnotationEngine

The AnnotationClient is used to interact with the AnnotationEngine service to create tables from existing schema, upload new data, and download existing annotations. Note that annotations in the AnnotationEngine are not linked to any particular segmentation, and thus do not include any root ids. An annotation client is accessed with `client.annotation`.

#### Get existing tables

A list of the existing tables for the dataset can be found at with `get_tables`.

In [25]:
all_tables = client.annotation.get_tables()
all_tables

[{'chunk_size': [512, 512, 64],
  'max_annotation_id': 124,
  'schema_name': 'microns_func_coreg',
  'table_name': 'ais_analysis_soma'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 312,
  'schema_name': 'microns_func_coreg',
  'table_name': 'ais_bounds'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 321,
  'schema_name': 'microns_func_coreg',
  'table_name': 'ais_bounds_v2'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 315,
  'schema_name': 'microns_func_coreg',
  'table_name': 'ais_bounds_v3'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 764,
  'schema_name': 'cell_type_local',
  'table_name': 'cell_type_ai_manual'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 298,
  'schema_name': 'plastic_synapse',
  'table_name': 'chandelier_plasticity'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 927,
  'schema_name': 'cell_type_local',
  'table_name': 'chc_input_valence'},
 {'chunk_size': [512, 512, 64],
  'max_annotation_id': 8623

Each table has three main properties that can be useful to know:
* `table_name` : The table name, used to refer to it when uploading or downloading annotations. This is also passed through to the table in the Materialized database.
* `schema_name` : The name of the table's schema from EMAnnotationSchemas (see below).
* `max_annotation_id` : An upper limit on the number of annotations already contained in the table.

#### Downloading annotations

You can download the JSON representation of a data point through the `get_annotation` method. This can be useful if you need to look up information on unmaterialized data, or to see what a properly templated annotation looks like.

In [26]:
table_name = all_tables[0]['table_name'] # 'ais_analysis_soma'
annotation_id = 100
client.annotation.get_annotation(annotation_id=annotation_id, table_name=table_name)

{'func_id': 0,
 'oid': '100',
 'pt': {'position': [111366, 62071, 1219]},
 'type': 'microns_func_coreg'}

#### Create a new table

One can create a new table with a specified schema with the `create_table` method:

```
client.annotation.create_table(table_name='test_table',
                               schema_name='microns_func_coreg')
```

Now, new data can be generated as a dict or list of dicts following the schema and uploaded with `post_annotation`.
For example, a `microns_func_coreg` point needs to have:
    * `type` set to `microns_func_coreg`
    * `pt` set to a dict with `position` as a key and the xyz location as a value.
    * `func_id` set to an integer.
    
The following will create a new annotation and upload it to the service: 
```
new_data = {'type': 'microns_func_coreg',
            'pt': {'position': [1,2,3]},
            'func_id': 0}
            
client.annotation.post_annotation(table_name='test_table', data=[new_data])
```

### 7. EMAnnotationSchemas

The EMAnnotationSchemas client lets one look up the available schemas and how they are defined. This is mostly used for programmatic interactions between services, but can be useful when looking up schema definitions for new tables.

#### Get the list of schema 
One can get the list of all available schema with the `schema` method. Currently, new schema have to be generated on the server side, although we aim to have a generic set available to use.

In [27]:
client.schema.schema()

['synapse',
 'presynaptic_bouton_type',
 'postsynaptic_compartment',
 'microns_func_coreg',
 'cell_type_local',
 'flat_segmentation_reference',
 'bound_tag',
 'extended_classical_cell_type',
 'plastic_synapse',
 'glia_contact']

#### View a specific schema

The details of each schema can be viewed with the `schema_definition` method, formatted as per JSONSchema. 

In [28]:
example_schema = client.schema.schema_definition('microns_func_coreg')
example_schema

{'$ref': '#/definitions/FunctionalCoregistration',
 'definitions': {'BoundSpatialPoint': {'properties': {'position': {'description': 'spatial position in voxels ofx,y,z of annotation',
     'items': {'format': 'integer', 'title': 'position', 'type': 'number'},
     'maxItems': 3,
     'minItems': 3,
     'postgis_geometry': 'POINTZ',
     'title': 'position',
     'type': 'array'},
    'root_id': {'type': 'integer'},
    'supervoxel_id': {'type': 'integer'}},
   'required': ['position'],
   'type': 'object'},
  'FunctionalCoregistration': {'properties': {'func_id': {'description': 'functional cell ID',
     'format': 'integer',
     'title': 'func_id',
     'type': 'number'},
    'pt': {'$ref': '#/definitions/BoundSpatialPoint',
     'description': 'location of cell body of functional cell',
     'type': 'object'},
    'type': {'description': 'type of annotation',
     'drop_column': True,
     'title': 'type',
     'type': 'string'},
    'valid': {'description': 'is this annotation va

This is mostly useful for programmatic interaction between services at the moment, but can also be used to inspect the expected form of an annotation by digging into the format.

In [29]:
example_schema['definitions']['FunctionalCoregistration']

{'properties': {'func_id': {'description': 'functional cell ID',
   'format': 'integer',
   'title': 'func_id',
   'type': 'number'},
  'pt': {'$ref': '#/definitions/BoundSpatialPoint',
   'description': 'location of cell body of functional cell',
   'type': 'object'},
  'type': {'description': 'type of annotation',
   'drop_column': True,
   'title': 'type',
   'type': 'string'},
  'valid': {'description': 'is this annotation valid',
   'title': 'valid',
   'type': 'boolean'}},
 'required': ['func_id', 'pt', 'type'],
 'type': 'object'}